# Figure 3 Circle Heatmap all trans Proteins

This notebook take lists of genes from the enrichment analysis and graphs the correlation in circle heat maps. 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math as math
import scipy.stats
import re
import sys 
import statsmodels.stats.multitest

import cptac
import cptac.utils as u
import plot_utils as p

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Complement Genes

Insert list from enrichment analysis and add back proteomics to name

In [2]:

complement_pos_neg = ['FGB','ITGAM','VWF','F10','SERPIND1','SERPINC1','PROS1','SERPINE1','PLAUR','F2','C8A','CLU','KNG1','C2','C3','PROCR','C5','C8G','C6','PROC','A2M','KLKB1','CFB']
complement_genes = ['CFD','CPB2','ITGAM','CFH','C1S','C1R','SERPINC1','PROS1','SERPINE1','ITGB2','F13A1','PLG','C8B','C8A','CLU','KNG1','C2','C3','THBD','C5','C8G','C6','C7','PLAU','VSIG4','A2M','FGB','FGA','CR2','SERPINB2','VWF','F10','FGG','SERPINF2','F11','PLAUR','F2','F3','PROCR','F9','PROC','SERPING1','F13B','KLKB1','CFB']
complement_column_names = []
for gene in complement_genes:
    gene += "_proteomics"
    complement_column_names.append(gene)
len(complement_column_names)


45

Load appended version of data frame with correlations with FDR correction. Subset down dataframe to only have complement genes

In [4]:

df_FDR_append = pd.read_csv("../Step3.2_combining_pearson_dfs/csv_files/pancan_EGFR_pearson_sig_all_prot_append_FDR.csv")
df_FDR_append = df_FDR_append.drop(['Unnamed: 0'], axis=1)

df_FDR_complement = df_FDR_append[df_FDR_append.Comparison.isin(complement_column_names)]
df_FDR_complement


,Comparison,Correlation,P_value,Cancer Type
77,PROCR_proteomics,-0.470784,8.763500e-07,GBM
196,PLAUR_proteomics,-0.425639,1.122876e-05,GBM
244,SERPINB2_proteomics,-0.482756,1.749352e-05,GBM
305,FGB_proteomics,-0.404936,3.218521e-05,GBM
350,FGG_proteomics,-0.396275,4.899957e-05,GBM
...,...,...,...,...
8091,C7_proteomics,0.324477,1.184867e-03,Colon
8166,A2M_proteomics,0.311803,1.877305e-03,Colon
8233,KNG1_proteomics,0.302283,2.618696e-03,Colon
8479,PLAU_proteomics,0.385665,3.170871e-05,Lscc



Make figure size of circles base on p_values and color based on correlation. Non sigificant correlations included. 

In [5]:

p.plotCircleHeatMap(df_FDR_complement, "P_value","Correlation","Comparison","Cancer Type")

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/plot_utils/__init__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["size2"] = df[circle_var].apply(lambda x: -1*(np.log(x)))
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/plot_utils/__init__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'] = (df["size2"])*3


Loading BokehJS ...

Make same graph, but filter out non significant values

In [9]:
df_FDR_append_sig_only = df_FDR_complement[df_FDR_complement["P_value"] < 0.005]

In [10]:
p.plotCircleHeatMap(df_FDR_append_sig_only,"P_value","Correlation","Comparison","Cancer Type")

Loading BokehJS ...

Load in wide data frame and subset with genes in figure in order to see raw numbers associated with figure 

In [26]:
df_FDR_wide = pd.read_csv("../Step3.2_combining_pearson_dfs/csv_files/pancan_EGFR_all_FDR_wide.csv")
df_FDR_wide = df_FDR_wide.drop(['Unnamed: 0'], axis=1)

df = df_FDR_wide.rename(columns={"Comparison": "Genes"})

df_FDR_wide_complement = df_FDR_wide[df_FDR_wide.Comparison.isin(complement_column_names)]

df = df.set_index('Genes')
df

,Correlation_Gbm,P_value_Gbm,Correlation_kidney,P_value_kidney,Correlation_Ovar,P_value_Ovar,Correlation_Brca,P_value_Brca,Correlation_Luad,P_value_Luad,Correlation_hnscc,P_value_hnscc,Correlation_colon,P_value_colon,Correlation_Lscc,P_value_Lscc
Genes,,,,,,,,,,,,,,,,
EGFR_proteomics,1.000000,0.000000e+00,1.0,0.0,1.0,0.0,1.000000,0.00000,1.000000,0.000000,1.000000,0.000000e+00,1.0,0.0,1.000000,0.000000e+00
PHLDA1_proteomics,0.816848,6.553435e-25,NaN,NaN,NaN,NaN,0.364797,0.00019,NaN,NaN,0.664271,3.426615e-15,NaN,NaN,0.709812,3.919195e-18
GRB2_proteomics,-0.610889,1.886384e-11,NaN,NaN,NaN,NaN,NaN,NaN,-0.302439,0.001321,-0.532341,2.559824e-09,NaN,NaN,NaN,NaN
SOCS2_proteomics,0.562720,1.343464e-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.478574,2.191662e-04
CDH4_proteomics,0.559180,1.790048e-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SLIT3_proteomics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.291391,2.011415e-03
AAK1_proteomics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.290954,2.044509e-03
COMMD3-BMI1_proteomics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.290800,2.056267e-03


# Endocytosis genes

In [175]:

endocytosis = ['SH3GLB1','ARFGEF1','PRKCI','SMAD3','WIPF2','CAV1','VPS4B','PARD6G','ARPC4','ARPC5','IGF1R','EPN3','ACAP2','DAB2','PARD6B','ZFYVE16','ERBB3','ARPC3','NEDD4','CAPZA1','KIT','CHMP4C','MET']
endocytosis_column_names = []
for gene in endocytosis:
    gene += "_proteomics"
    endocytosis_column_names.append(gene)
len(endocytosis_column_names)


23

In [176]:
df_FDR_endocytosis = df_FDR_append[df_FDR_append.Comparison.isin(endocytosis_column_names)]
df_FDR_endocytosis

,Comparison,Correlation,P_value,Cancer Type
6,DAB2_proteomics,-0.556402,2.237010e-09,GBM
164,NEDD4_proteomics,-0.434925,6.843176e-06,GBM
206,ARPC3_proteomics,-0.423330,1.267190e-05,GBM
291,CAV1_proteomics,-0.407997,2.766397e-05,GBM
399,CAPZA1_proteomics,-0.387765,7.323051e-05,GBM
...,...,...,...,...
75498,ARPC4_proteomics,0.071803,4.560076e-01,Lscc
75838,DAB2_proteomics,-0.066031,4.931051e-01,Lscc
76307,ERBB3_proteomics,-0.057605,5.500006e-01,Lscc
77330,ARPC3_proteomics,0.039662,6.807908e-01,Lscc


In [177]:
plotCircleHeatMap(df_FDR_endocytosis,"P_value","Correlation","Comparison","Cancer Type")

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

Loading BokehJS ...

In [178]:
df_FDR_wide_endocytosis = df_FDR_wide[df_FDR_wide.Comparison.isin(endocytosis_column_names)]
df_FDR_wide_endocytosis

,Comparison,Correlation_Gbm,P_value_Gbm,Correlation_kidney,P_value_kidney,Correlation_Ovar,P_value_Ovar,Correlation_Brca,P_value_Brca,Correlation_Luad,P_value_Luad,Correlation_hnscc,P_value_hnscc,Correlation_colon,P_value_colon,Correlation_Lscc,P_value_Lscc
6,DAB2_proteomics,-0.556402,2.237010e-09,NaN,NaN,NaN,NaN,0.326055,3.764707e-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164,NEDD4_proteomics,-0.434925,6.843176e-06,0.321219,0.000622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206,ARPC3_proteomics,-0.423330,1.267190e-05,NaN,NaN,NaN,NaN,0.311784,6.939810e-04,-0.301699,1.359677e-03,NaN,NaN,NaN,NaN,NaN,NaN
291,CAV1_proteomics,-0.407997,2.766397e-05,0.344889,0.000224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
399,CAPZA1_proteomics,-0.387765,7.323051e-05,NaN,NaN,NaN,NaN,0.311677,6.970752e-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
449,ZFYVE16_proteomics,-0.379396,1.075736e-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.390961,0.000075,NaN,NaN
472,SH3GLB1_proteomics,-0.375266,1.295641e-04,NaN,NaN,NaN,NaN,0.316491,5.690949e-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
601,ARPC4_proteomics,-0.357529,2.800944e-04,-0.264291,0.005270,NaN,NaN,0.354157,1.031524e-04,-0.294842,1.767048e-03,NaN,NaN,NaN,NaN,NaN,NaN
662,ARPC5_proteomics,-0.349300,3.946124e-04,NaN,NaN,NaN,NaN,0.353939,1.042416e-04,-0.270303,4.290947e-03,NaN,NaN,NaN,NaN,NaN,NaN
697,MET_proteomics,-0.344604,4.779015e-04,NaN,NaN,NaN,NaN,0.451064,7.889147e-05,0.485692,7.518188e-08,NaN,NaN,NaN,NaN,NaN,NaN


# Aktin Genes

In [179]:
aktin_genes = ['ITGB1','GSN','ITGAM','ITGB5','ITGA3','ITGB3','LIMK2','ARPC4','ARPC5','F2','BAIAP2','ARHGAP35','NRAS','DIAPH2','ARPC3','ARHGEF1','PFN1','VCL']

In [180]:
aktin_column_names = []
for gene in aktin_genes:
    gene += "_proteomics"
    aktin_column_names.append(gene)
len(aktin_column_names)

18

In [181]:
df_FDR_aktin = df_FDR_append[df_FDR_append.Comparison.isin(aktin_column_names)]
df_FDR_aktin

,Comparison,Correlation,P_value,Cancer Type
206,ARPC3_proteomics,-0.423330,0.000013,GBM
271,ITGB1_proteomics,-0.412364,0.000022,GBM
315,PFN1_proteomics,-0.402695,0.000036,GBM
526,VCL_proteomics,-0.368318,0.000176,GBM
601,ARPC4_proteomics,-0.357529,0.000280,GBM
...,...,...,...,...
75970,GSN_proteomics,0.077248,0.510065,Lscc
76446,NRAS_proteomics,-0.054907,0.568870,Lscc
77330,ARPC3_proteomics,0.039662,0.680791,Lscc
79285,VCL_proteomics,0.008139,0.932744,Lscc


In [182]:
plotCircleHeatMap(df_FDR_aktin,"P_value","Correlation","Comparison","Cancer Type")

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

Loading BokehJS ...

In [29]:
df_FDR_wide_T = df.T 
df_FDR_wide_T

df_FDR_wide_T.filter(like='CDH1', axis=1)

Genes,PCDH17_proteomics,PCDH19_proteomics,CDH10_proteomics,CDH13_proteomics,CDH1_proteomics,CDH17_proteomics
Correlation_Gbm,5.060447e-01,0.437346,0.296208,NaN,NaN,NaN
P_value_Gbm,9.126316e-08,0.000006,0.002912,NaN,NaN,NaN
Correlation_kidney,NaN,NaN,NaN,NaN,NaN,NaN
P_value_kidney,NaN,NaN,NaN,NaN,NaN,NaN
Correlation_Ovar,NaN,NaN,NaN,NaN,NaN,NaN
P_value_Ovar,NaN,NaN,NaN,NaN,NaN,NaN
Correlation_Brca,NaN,NaN,NaN,0.390674,NaN,NaN
P_value_Brca,NaN,NaN,NaN,0.000016,NaN,NaN
Correlation_Luad,NaN,NaN,NaN,NaN,0.299164,-0.475269
P_value_Luad,NaN,NaN,NaN,NaN,0.001499,0.003398


# Hemostasis Genes

In [72]:
hemostasis = ['ITGB1','ITIH4','ITGAM','SERPINC1','DGKB','DGKA','PROS1','ITGB3','SERPINE1','ATP2A3','ITPR3','A1BG','RASGRP2','CLU','KNG1','NRAS','GNG2','GNG7','KIF13B','QSOX1','A2M','FGB','CXADR','SERPIND1','F10','VWF','PRKCB','ITGA3','CAV1','PLAUR','PLA2G4A','PPP2R5A','ATP1B3','F2','SERPINB8','APBB1IP','PROCR','PROC','CAPZA1','PECAM1','PFN1','KLKB1','VCL']


In [73]:
hemo_column_names = []
for gene in hemostasis:
    gene += "_proteomics"
    hemo_column_names.append(gene)
len(hemo_column_names)

43

In [74]:
df_FDR_hemo = df_FDR_append[df_FDR_append.Comparison.isin(hemo_column_names)]
df_FDR_hemo

,Comparison,Correlation,P_value,Cancer Type
53,APBB1IP_proteomics,-0.483056,4.103970e-07,GBM
77,PROCR_proteomics,-0.470784,8.763500e-07,GBM
196,PLAUR_proteomics,-0.425639,1.122876e-05,GBM
198,SERPINB8_proteomics,-0.425614,1.124355e-05,GBM
271,ITGB1_proteomics,-0.412364,2.223482e-05,GBM
...,...,...,...,...
77647,CLU_proteomics,-0.034342,7.217108e-01,Lscc
77872,SERPINC1_proteomics,-0.030863,7.489112e-01,Lscc
78145,QSOX1_proteomics,0.026299,7.850674e-01,Lscc
79271,A2M_proteomics,0.008291,9.314930e-01,Lscc


In [75]:
plotCircleHeatMap(df_FDR_hemo,"P_value","Correlation","Comparison","Cancer Type")

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

Loading BokehJS ...

In [76]:
df_FDR_append_sig_only = df_FDR_hemo[df_FDR_hemo["P_value"] < 0.005]

In [77]:
plotCircleHeatMap(df_FDR_append_sig_only,"P_value","Correlation","Comparison","Cancer Type")

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

Loading BokehJS ...

In [91]:
df_FDR_wide_hemo = df_FDR_wide[df_FDR_wide.Comparison.isin(hemo_column_names)]
df_FDR_wide_hemo

,Comparison,Correlation_Gbm,P_value_Gbm,Correlation_kidney,P_value_kidney,Correlation_Ovar,P_value_Ovar,Correlation_Brca,P_value_Brca,Correlation_Luad,P_value_Luad,Correlation_hnscc,P_value_hnscc,Correlation_colon,P_value_colon,Correlation_Lscc,P_value_Lscc
53,APBB1IP_proteomics,-0.483056,4.103970e-07,NaN,NaN,NaN,NaN,NaN,NaN,-0.277151,3.376269e-03,-0.382555,4.039015e-05,0.404376,0.000040,NaN,NaN
77,PROCR_proteomics,-0.470784,8.763500e-07,0.284925,0.002553,NaN,NaN,0.336022,2.412023e-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,PLAUR_proteomics,-0.425639,1.122876e-05,NaN,NaN,NaN,NaN,0.467428,1.389846e-07,NaN,NaN,0.278504,3.363640e-03,NaN,NaN,NaN,NaN
198,SERPINB8_proteomics,-0.425614,1.124355e-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.430563,0.000011,NaN,NaN
271,ITGB1_proteomics,-0.412364,2.223482e-05,0.276027,0.003513,NaN,NaN,0.410530,5.191577e-06,NaN,NaN,0.347360,2.150377e-04,NaN,NaN,NaN,NaN
291,CAV1_proteomics,-0.407997,2.766397e-05,0.344889,0.000224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305,FGB_proteomics,-0.404936,3.218521e-05,NaN,NaN,NaN,NaN,0.303047,9.944820e-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
315,PFN1_proteomics,-0.402695,3.592363e-05,NaN,NaN,NaN,NaN,0.303084,9.929697e-04,-0.370262,6.849871e-05,NaN,NaN,0.332115,0.000889,NaN,NaN
348,DGKA_proteomics,-0.396497,4.848305e-05,-0.324800,0.000536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.363821,0.000093
399,CAPZA1_proteomics,-0.387765,7.323051e-05,NaN,NaN,NaN,NaN,0.311677,6.970752e-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
immune_sys = ['ITGB1','LRSAM1','ITGAM','AHCYL1','ITGB5','PROS1','WIPF2','ITGB3','NCF4','ICAM3','PTPRJ','IFI30','CLU','C8A','PVR','PSTPIP1','ACTR1A','C8G','CASP10','MRC1','FBXO6','LBP','KPNA3','CTSC','CTSB','FGB','ACTR2','CXADR','VWF','PRKCB','PPP2R5A','IL16','GFRA1','CYBA','TMEM173','APBB1IP','HCK','KIT','RAPGEF2','BPI','CFB','VCL','TLR2','UBA7','ITPR3','BCL10','RASAL3','RASGRP2','MALT1','C2','C3','C5','NRAS','C6','ERBB3','STAT6','FLNB','GBP4','AP1M2','LAIR1','VASP','ARPC4','ARPC5','BAIAP2','LILRB4','NFKB2','ISG20','FCGR2A','ARPC3','NEDD4','CAPZA1','REL','STUB1','FCGR2B','PIK3AP1','PTPN2','LGMN']

In [79]:
immune_column_names = []
for gene in immune_sys:
    gene += "_proteomics"
    immune_column_names.append(gene)
len(immune_column_names)

77

In [80]:
df_FDR_immune = df_FDR_append[df_FDR_append.Comparison.isin(immune_column_names)]
df_FDR_immune

,Comparison,Correlation,P_value,Cancer Type
8,CTSC_proteomics,-0.546285,4.951742e-09,GBM
33,CTSB_proteomics,-0.496895,1.682242e-07,GBM
53,APBB1IP_proteomics,-0.483056,4.103970e-07,GBM
93,RASAL3_proteomics,-0.464579,1.271949e-06,GBM
99,REL_proteomics,-0.461185,1.554594e-06,GBM
...,...,...,...,...
78336,GBP4_proteomics,0.023132,8.104270e-01,Lscc
78502,FBXO6_proteomics,0.020502,8.316445e-01,Lscc
79285,VCL_proteomics,0.008139,9.327439e-01,Lscc
79363,ACTR2_proteomics,0.006766,9.440729e-01,Lscc


In [86]:
plotCircleHeatMap(df_FDR_immune,"P_value","Correlation","Comparison","Cancer Type")

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

Loading BokehJS ...

In [88]:
df_FDR_append_sig_only = df_FDR_immune[df_FDR_immune["P_value"] < 0.005]

In [89]:
plotCircleHeatMap(df_FDR_append_sig_only,"P_value","Correlation","Comparison","Cancer Type")

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

Loading BokehJS ...